# 기초통계

In [ ]:
!sudo apt-get install -y fonts-nanum
!sudo fc-cache -fv
!rm ~/.cache/matplotlib -rf

Reading package lists... Done
Building dependency tree       
Reading state information... Done
fonts-nanum is already the newest version (20170925-1).
0 upgraded, 0 newly installed, 0 to remove and 10 not upgraded.
/usr/share/fonts: caching, new cache contents: 0 fonts, 1 dirs
/usr/share/fonts/truetype: caching, new cache contents: 0 fonts, 3 dirs
/usr/share/fonts/truetype/humor-sans: caching, new cache contents: 1 fonts, 0 dirs
/usr/share/fonts/truetype/liberation: caching, new cache contents: 16 fonts, 0 dirs
/usr/share/fonts/truetype/nanum: caching, new cache contents: 10 fonts, 0 dirs
/usr/local/share/fonts: caching, new cache contents: 0 fonts, 0 dirs
/root/.local/share/fonts: skipping, no such directory
/root/.fonts: skipping, no such directory
/var/cache/fontconfig: cleaning cache directory
/root/.cache/fontconfig: not cleaning non-existent cache directory
/root/.fontconfig: not cleaning non-existent cache directory
fc-cache: succeeded


In [ ]:
import warnings
warnings.filterwarnings("ignore")

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA, TruncatedSVD
import matplotlib.patches as mpatches
import time

# Classifier Libraries
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.svm import LinearSVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import RobustScaler
from xgboost import XGBClassifier
import xgboost

import collections

# Other Libraries
from sklearn.model_selection import train_test_split
from imblearn.pipeline import Pipeline
from sklearn.pipeline import make_pipeline
from imblearn.pipeline import make_pipeline as imbalanced_make_pipeline
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import NearMiss
from imblearn.metrics import classification_report_imbalanced
from sklearn.metrics import precision_score, recall_score, f1_score, fbeta_score, roc_auc_score, accuracy_score, classification_report, auc
from sklearn.metrics import make_scorer
from sklearn.metrics import average_precision_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import mean_squared_error
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
from sklearn.metrics import precision_recall_curve
from collections import Counter
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.model_selection import StratifiedShuffleSplit

 
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV

from itertools import chain

In [ ]:
pd_train=pd.read_csv('/content/train.csv', na_values="na")
pd_train

dt_of_fr fr_yn  ... blk_dngrs_thng_mnfctr_yn cltrl_hrtg_yn
0      2017-10-20 05:54     Y  ...                      NaN           NaN
1      2018-09-30 08:26     N  ...                      NaN           NaN
2      2016-10-30 14:57     Y  ...                      NaN           NaN
3      2016-06-14 05:23     N  ...                      NaN           NaN
4      2018-04-22 05:38     N  ...                      NaN           NaN
...                 ...   ...  ...                      ...           ...
59194  2016-12-03 14:25     N  ...                        N             N
59195  2016-10-29 10:10     N  ...                        N             N
59196  2018-03-03 19:49     N  ...                        N             N
59197  2017-09-27 11:10     N  ...                        N             N
59198  2015-09-20 07:34     N  ...                        N             N

[59199 rows x 180 columns]

In [ ]:
#gas~, ele~, Iw~에 해당되는 변수들을 제외하고 어떤 변수들이 있는지 알아보자.

non_series_item=[]
series_item=[]

for item in pd_train.columns:
  if item[0:3]=="gas" or item[0:3]=="ele" or item[0:3]=="lw_":
    series_item.append(item)
  else:
    non_series_item.append(item)

In [ ]:
for i in non_series_item:
  print(i)

dt_of_fr
fr_yn
bldng_us
bldng_archtctr
bldng_cnt
bldng_ar
ttl_ar
lnd_ar
dt_of_athrztn
ttl_grnd_flr
ttl_dwn_flr
bldng_us_clssfctn
tmprtr
prcpttn
wnd_spd
wnd_drctn
hmdt
jmk
id
rgnl_ar_nm
rgnl_ar_nm2
lnd_us_sttn_nm
rd_sd_nm
emd_nm
hm_cnt
fr_sttn_dstnc
bldng_ar_prc
fr_wthr_fclt_dstnc
fr_mn_cnt
mlt_us_yn
cctv_dstnc
fr_wthr_fclt_in_100m
cctv_in_100m
tbc_rtl_str_dstnc
sft_emrgnc_bll_dstnc
ahsm_dstnc
no_tbc_zn_dstnc
bldng_cnt_in_50m
trgt_crtr
fr_fghtng_fclt_spcl_css_5_yn
fr_fghtng_fclt_spcl_css_6_yn
us_yn
dngrs_thng_yn
slf_fr_brgd_yn
blk_dngrs_thng_mnfctr_yn
cltrl_hrtg_yn


태욱 전처리

In [ ]:
#사후변수거나 데이터 예측에 상관없는 변수들
"""
id    아이디(분석에 미사용) 
dt_of_fr    화재발생일시
fr_yn    화재발생여부
bldng_us    건물용도
bldng_archtctr    건물구조
bldng_cnt    건물채수
bldng_ar    건물건축면적
ttl_ar    건물연면적(건물층별합계전체 면적)
lnd_ar    토지면적
dt_of_arthrztn    건물승인일자
ttl_grnd_flr    건물들의 지상 층수의 합
ttl_dwn_fr    건물들의 지하 층수의 합
bldng_us_clssfctn    건물용도분류명
tmprtr    온도(c)
prcpttn    강수량
wnd_spd    풍속
wnd_drctn    풍향
hmdt    습도
ele_engry_us_201401    전기 에너지 사용량(2014월1월)
ele_engry_us_201812    전기 에너지 사용량(2018월12월)
gas_engry_us_201401    가스 에너지 사용량(2014월1월)
gas_engry_us_201812    가스 에너지 사용량(2018월12월)
lw_13101010    복도/계단/출입구의 성능 유지여부(0~5)
lw_13101110    옥상광장의 피난성능 유지여부(0~5)
lw_13101210    방화문/방화셔터 등의 성능 유지여부(0~5)
lw_13101211    방화구획 적합 여부(0~5)
lw_13101310    경계벽 및 칸막이벽의 변경 등 방화성능 유지여부(0~5)
lw_13101410    배연설비의 성능 유지여부(0~5)
lw_13111010    내화구조의 성능 유지여부(0~5)
lw_13111110    방화벽의 성능 유지여부(0~5)
lw_13121010    외벽의 성능 유지여부(0~5)
lw_13121011    창호의 성능 유지여부(0~5)
lw_13131010    내부마감의 방화성능 유지여부(0~5)
lw_13131110    외부마감의 노후화 및 마감재 탈락 여부(0~5)
lw_13141010    지하층의 소방설비 성능 유지여부(0~5)
lw_13141011    지하층 피난구/피난계단의 성능 유지여부(0~5)
jmk    지적상 지목
rgnl_ar_nm    용도지역지구명
rgnl_ar_nm2    용도지역지구명2
lnd_us_sttn_nm    토지이용상황명
rd_sd_nm    도로측면명
emd_nm    행정구역명
hm_cnt    행정구역 인구
fr_sttn_dstnc    119 안전센터와의 거리
bldng_ar_prc    단위 면적당 건물 가격(2019년)
fr_wthr_fclt_dstnc    소방용수시설(소화전 등)과의 최소 거리
fr_mn_cnt    관할 소방서 인원
mlt_us_yn    다중이용시설 포함여부
cctv_dstnc    공공 CCTV와의 최소 거리
cctv_in_100m    반경 100m 이내 공공 CCTV
fr_wthr_fclt_in_100m    반경 100m 이내 소방용수 시설 수
tbc_rtl_str_dstnc    담배 소매점과의 최소 거리
sft_emrgnc_bll_dstnc    안전 비상벨과의 최소 거리
ahsm_dstnc    자동 심장 충격기와의 최소 거리
no_tbc_zn_dstnc    금연구역과의 최소 거리
trgt_crtr    소방관리대상물기준
fr_fghtng_fclt_spcl_css_5_yn    소방시설특례5호여부
fr_fghtng_fclt_spcl_css_6_yn    소방시설특례6호여부
us_yn    사용여부
dngrs_thng_yn    위험물대상여부
slf_fr_brgd_yn    자체소방대여부
blk_dngrs_thng_mnfctr_yn    대량위험물제조소등여부
cltrl_hrtg_yn    문화재여부
"""

'\nid    아이디(분석에 미사용) \ndt_of_fr    화재발생일시\nfr_yn    화재발생여부\nbldng_us    건물용도\nbldng_archtctr    건물구조\nbldng_cnt    건물채수\nbldng_ar    건물건축면적\nttl_ar    건물연면적(건물층별합계전체 면적)\nlnd_ar    토지면적\ndt_of_arthrztn    건물승인일자\nttl_grnd_flr    건물들의 지상 층수의 합\nttl_dwn_fr    건물들의 지하 층수의 합\nbldng_us_clssfctn    건물용도분류명\ntmprtr    온도(c)\nprcpttn    강수량\nwnd_spd    풍속\nwnd_drctn    풍향\nhmdt    습도\nele_engry_us_201401    전기 에너지 사용량(2014월1월)\nele_engry_us_201812    전기 에너지 사용량(2018월12월)\ngas_engry_us_201401    가스 에너지 사용량(2014월1월)\ngas_engry_us_201812    가스 에너지 사용량(2018월12월)\nlw_13101010    복도/계단/출입구의 성능 유지여부(0~5)\nlw_13101110    옥상광장의 피난성능 유지여부(0~5)\nlw_13101210    방화문/방화셔터 등의 성능 유지여부(0~5)\nlw_13101211    방화구획 적합 여부(0~5)\nlw_13101310    경계벽 및 칸막이벽의 변경 등 방화성능 유지여부(0~5)\nlw_13101410    배연설비의 성능 유지여부(0~5)\nlw_13111010    내화구조의 성능 유지여부(0~5)\nlw_13111110    방화벽의 성능 유지여부(0~5)\nlw_13121010    외벽의 성능 유지여부(0~5)\nlw_13121011    창호의 성능 유지여부(0~5)\nlw_13131010    내부마감의 방화성능 유지여부(0~5)\nlw_13131110    외부마감의 노후화 및 마감재 탈락 여부

In [ ]:
#np.nan값이 너무 큰 것은 다 날리자.

In [ ]:
"""
wnd_drctn    풍향
hmdt    습도
ele_engry_us_201401    전기 에너지 사용량(2014월1월) 
ele_engry_us_201812    전기 에너지 사용량(2018월12월) 
gas_engry_us_201401    가스 에너지 사용량(2014월1월) 
gas_engry_us_201812    가스 에너지 사용량(2018월12월) 
lw_13101010    복도/계단/출입구의 성능 유지여부(0~5) 
"""

'\nwnd_drctn    풍향\nhmdt    습도\nele_engry_us_201401    전기 에너지 사용량(2014월1월) \nele_engry_us_201812    전기 에너지 사용량(2018월12월) \ngas_engry_us_201401    가스 에너지 사용량(2014월1월) \ngas_engry_us_201812    가스 에너지 사용량(2018월12월) \nlw_13101010    복도/계단/출입구의 성능 유지여부(0~5) \n'

In [ ]:
pd_train[['wnd_drctn','hmdt','ele_engry_us_201401','ele_engry_us_201812','gas_engry_us_201401','gas_engry_us_201812','lw_13101010']].isnull().sum()

wnd_drctn                221
hmdt                      22
ele_engry_us_201401    27581
ele_engry_us_201812    27581
gas_engry_us_201401    27581
gas_engry_us_201812    27581
lw_13101010            58897
dtype: int64

In [ ]:
print(58897/len(pd_train)) ##99%가 결측치.... 변수 삭제

0.9948985624757175


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

plt.figure(figsize=(13,10))

plt.subplot(221)
sns.distplot(pd_train['ele_engry_us_201401'])

plt.subplot(222)
sns.distplot(pd_train['ele_engry_us_201812'])

plt.subplot(223)
sns.distplot(pd_train['gas_engry_us_201401'])

plt.subplot(224)
sns.distplot(pd_train['gas_engry_us_201812'])

plt.show()

In [ ]:
pd_train['ele_engry_us_201401'].value_counts()

0.0        31189
4095.0         3
105.0          3
3318.0         3
7203.0         3
           ...  
95382.0        1
19755.0        1
35454.0        1
1440.0         1
69657.0        1
Name: ele_engry_us_201401, Length: 380, dtype: int64

In [ ]:
pd_elenew=pd_train['ele_engry_us_201401'][pd_train['ele_engry_us_201401']!=0]
pd_elenew.value_counts()

4095.0      3
273.0       3
105.0       3
3318.0      3
7203.0      3
           ..
11592.0     1
337500.0    1
2125.0      1
2856.0      1
2751.0      1
Name: ele_engry_us_201401, Length: 379, dtype: int64

In [ ]:
pd_train['gas_engry_us_201401'].value_counts()

0.0           31332
25665200.0        5
50752.0           4
987056.0          4
487380.0          3
              ...  
3500.0            1
16284.0           1
2004.0            1
27965.0           1
127194.0          1
Name: gas_engry_us_201401, Length: 254, dtype: int64

In [ ]:
pd_gasnew=pd_train['gas_engry_us_201401'][pd_train['gas_engry_us_201401']!=0]
pd_gasnew.value_counts()

25665200.0    5
50752.0       4
987056.0      4
592983.0      3
138300.0      3
             ..
6310.0        1
2790.0        1
16201.0       1
3410.0        1
288.0         1
Name: gas_engry_us_201401, Length: 253, dtype: int64

In [ ]:
pd_gasnew=pd_gasnew.dropna(axis=0)

In [ ]:
pd_gasnew.count()
"""
총 59199 데이터중
0인 값 31332
nan 값 27581
실제값 253
"""

'\n총 59199 데이터중\n0인 값 31332\nnan 값 27581\n실제값 253\n'

In [ ]:
pd_gasnew.sort_values(ascending=True)

45890          142.0
32367          144.0
38613          156.0
32624          180.0
36221          180.0
            ...     
12658     25665200.0
12664     25665200.0
12786     25665200.0
12637     25665200.0
54945    215490912.0
Name: gas_engry_us_201401, Length: 286, dtype: float64

In [ ]:
temp_list=[]
for item in pd_gasnew:
  temp_list.append(item)
temp_list.sort()
for item in temp_list:
  print(item)

142.0
144.0
156.0
180.0
180.0
240.0
276.0
288.0
432.0
432.0
468.0
560.0
780.0
905.0
996.0
1128.0
1190.0
1260.0
1284.0
1337.0
1368.0
1440.0
1620.0
1660.0
1738.0
1846.0
1965.0
2004.0
2192.0
2205.0
2254.0
2354.0
2354.0
2380.0
2790.0
3040.0
3120.0
3222.0
3410.0
3500.0
3764.0
3836.0
3842.0
4008.0
4068.0
4191.0
4320.0
4384.0
4430.0
4530.0
4646.0
5070.0
5120.0
5500.0
5580.0
6118.0
6310.0
6432.0
6864.0
6952.0
7056.0
7182.0
7325.0
7860.0
8000.0
8588.0
8680.0
8768.0
9112.0
9144.0
9750.0
9855.0
10080.0
10219.0
10314.0
10319.0
10836.0
11070.0
11232.0
11533.0
11792.0
11808.0
11998.0
12342.0
13284.0
13336.0
13362.0
13961.0
14014.0
14478.0
14547.0
15036.0
15180.0
16201.0
16283.0
16284.0
16592.0
16722.0
16768.0
16785.0
17010.0
17176.0
17408.0
17500.0
17500.0
17612.0
17651.0
17654.0
17712.0
17904.0
18088.0
18544.0
19012.0
19056.0
19434.0
19616.0
19812.0
20043.0
20192.0
20824.0
20960.0
21222.0
21261.0
21344.0
21344.0
21960.0
22260.0
22330.0
22672.0
22865.0
24444.0
24662.0
25312.0
25346.0
25365.0
25536.0

In [ ]:
plt.figure(figsize=(13,10))

plt.subplot(121)
pd_elenew.hist()
plt.xlabel('electricity')

plt.subplot(122)
pd_gasnew.hist()
plt.xlabel('gas')

plt.tight_layout()

In [ ]:
plt.figure(figsize=(30,10))

plt.subplot(121)
sns.distplot(pd_train['wnd_drctn']) #풍향

plt.subplot(122)
sns.distplot(pd_train['hmdt']) #습도
plt.show()

In [ ]:
"""
전처리 들어가야할 부분
=================
dt_of_fr를 분기별로 one-hot encoding
lw_~ 바이너리로 one-hot encoding
=================
카테고리화 되어 있는 부분

bldng_us
bldng_archtctr
bldng_us_clssfctn
jmk
rgnl_ar_nm
rgnl_ar_nm2
lnd_us_sttn_nm	
rd_sd_nm
emd_nm

"""

'\n전처리 들어가야할 부분\n=================\ndt_of_fr를 분기별로 one-hot encoding\nlw_~ 바이너리로 one-hot encoding\n=================\n카테고리화 되어 있는 부분\n\nbldng_us\nbldng_archtctr\nbldng_us_clssfctn\njmk\nrgnl_ar_nm\nrgnl_ar_nm2\nlnd_us_sttn_nm\t\nrd_sd_nm\nemd_nm\n\n'

In [ ]:
from datetime import datetime

for i in range(len(pd_train['dt_of_fr'])): 
  #print(i)
  if pd_train['dt_of_fr'][i][5:7]=='12' or pd_train['dt_of_fr'][i][5:7]=='01' or pd_train['dt_of_fr'][i][5:7]=='02':
    pd_train['dt_of_fr'][i]=0 #겨울을 0으로 원핫
  elif pd_train['dt_of_fr'][i][5:7]=='03' or pd_train['dt_of_fr'][i][5:7]=='04' or pd_train['dt_of_fr'][i][5:7]=='05':
    pd_train['dt_of_fr'][i]=1 #봄을 1으로 원핫
  elif pd_train['dt_of_fr'][i][5:7]=='06' or pd_train['dt_of_fr'][i][5:7]=='07' or pd_train['dt_of_fr'][i][5:7]=='08':
    pd_train['dt_of_fr'][i]=2 #여름을 2으로 원핫
    #print(i)
  else:
    pd_train['dt_of_fr'][i]=3 #가을을 3으로 원핫"""

In [ ]:
""" pd_train['dt_of_athrztn'] = pd_train['dt_of_athrztn'].fillna(9999)

for i in range(len(pd_train['dt_of_athrztn'])): 
  #print(i)
  if pd_train['dt_of_athrztn'][i] == 9999 :
    pd_train['dt_of_athrztn'][i]= 0 #결측치를 0으로 원핫
  elif len(str(pd_train['dt_of_athrztn'][i])) == 4 and int(pd_train['dt_of_athrztn'][i]) < 1999 :
    pd_train['dt_of_athrztn'][i]=1 #오래된 건물을 1로 원핫
  elif len(str(pd_train['dt_of_athrztn'][i])) == 8 and int(str(pd_train['dt_of_athrztn'][i])[0:4]) < 1999 :
    pd_train['dt_of_athrztn'][i]=1 #오래된 건물을 1로 원핫
  else:
    pd_train['dt_of_athrztn'][i]=0 #해석 어려운 행을 0으로 원핫  """

" pd_train['dt_of_athrztn'] = pd_train['dt_of_athrztn'].fillna(9999)\n\nfor i in range(len(pd_train['dt_of_athrztn'])): \n  #print(i)\n  if pd_train['dt_of_athrztn'][i] == 9999 :\n    pd_train['dt_of_athrztn'][i]= 0 #결측치를 0으로 원핫\n  elif len(str(pd_train['dt_of_athrztn'][i])) == 4 and int(pd_train['dt_of_athrztn'][i]) < 1999 :\n    pd_train['dt_of_athrztn'][i]=1 #오래된 건물을 1로 원핫\n  elif len(str(pd_train['dt_of_athrztn'][i])) == 8 and int(str(pd_train['dt_of_athrztn'][i])[0:4]) < 1999 :\n    pd_train['dt_of_athrztn'][i]=1 #오래된 건물을 1로 원핫\n  else:\n    pd_train['dt_of_athrztn'][i]=0 #해석 어려운 행을 0으로 원핫  "

In [ ]:
#pd_train['dt_of_athrztn']

In [ ]:
LW_dict=[]

for item in pd_train.columns:
  if item[0:3]=="lw_":
    LW_dict.append(item)
  else:
    pass
LW_dict

['lw_13101010',
 'lw_13101110',
 'lw_13101210',
 'lw_13101211',
 'lw_13101310',
 'lw_13101410',
 'lw_13111010',
 'lw_13111110',
 'lw_13121010',
 'lw_13121011',
 'lw_13131010',
 'lw_13131110',
 'lw_13141010',
 'lw_13141011']

In [ ]:
#class가 lw_~로 시작하는 모든 column들 중에서 np.nan이나 0값인 원소들은 0으로, 0보다 큰 값을 가진 원소들은 1로 원핫 인코딩으로 한다.
for i in LW_dict:
  for j in range(len(pd_train[i])):
    if pd_train[i][j]>float(0): 
      pd_train[i][j]=1
    else:
      pd_train[i][j]=0

In [ ]:
#카테고리컬한 column들은 모두 제거한다.
categorical_list=['bldng_us','bldng_archtctr','bldng_us_clssfctn','jmk','rgnl_ar_nm','rgnl_ar_nm2','lnd_us_sttn_nm','rd_sd_nm','emd_nm','trgt_crtr','mlt_us_yn']
pd_train=pd_train.drop(columns=categorical_list,axis=1)

#결측치 값들이 너무 많은 column들/(lw_~로 시작하는 column들은 제외)들도 모두 제거한다.
outofdata_list=['dt_of_athrztn', 'fr_fghtng_fclt_spcl_css_5_yn','fr_fghtng_fclt_spcl_css_6_yn','us_yn','dngrs_thng_yn','slf_fr_brgd_yn','blk_dngrs_thng_mnfctr_yn','cltrl_hrtg_yn']
pd_train=pd_train.drop(columns=outofdata_list,axis=1)
#id 제거
pd_train=pd_train.drop(['id'], axis=1)
pd_train

dt_of_fr fr_yn  bldng_cnt  ...  ahsm_dstnc  no_tbc_zn_dstnc  bldng_cnt_in_50m
0            3     Y          3  ...       11322               88                 0
1            3     N          3  ...        3369             7727                 0
2            3     Y          1  ...       12451               72                14
3            2     N          2  ...         407              508                11
4            1     N          2  ...       12487              707                 0
...        ...   ...        ...  ...         ...              ...               ...
59194        0     N          2  ...        6679              914                 0
59195        3     N         35  ...        1285              131                 0
59196        1     N          2  ...        4878              790                 0
59197        3     N          2  ...        3572              613                 0
59198        3     N          3  ...       11769              415                 0

[59199 rows x 160 columns]

In [ ]:
##일단 임시로 nan값에 모두 0을 집어넣음
pd_train = pd_train.fillna(0)

## **undersampling/oversampling 구현**

In [ ]:
#over-sampling과 under-sampling

In [ ]:
화재_yes=0
화재_no=0
for item in pd_train['fr_yn']:
  if item=="Y":
    화재_yes+=1
  else:
    화재_no+=1
print("화재가 난 비율 {0:.2f}%, 화재가 안 난 비율 {1:.2f}%".format((화재_yes/len(pd_train['fr_yn']))*100,(화재_no/len(pd_train['fr_yn']))*100))

화재가 난 비율 12.93%, 화재가 안 난 비율 87.07%


In [ ]:
##데이터의 불균형이 존재

In [ ]:
pd_train_fireyes=pd_train[pd_train['fr_yn']=='Y']
pd_train_fireno=pd_train[pd_train['fr_yn']=='N']

In [ ]:
len(pd_train_fireyes)

7657

In [ ]:
"""#oversampling
import random
randomlist = []
for i in range(0,int(len(pd_train_fireno)-len(pd_train_fireyes))):
  n = random.randint(1,len(pd_train_fireyes))
  randomlist.append(n)

print(randomlist) """

'#oversampling\nimport random\nrandomlist = []\nfor i in range(0,int(len(pd_train_fireno)-len(pd_train_fireyes))):\n  n = random.randint(1,len(pd_train_fireyes))\n  randomlist.append(n)\n\nprint(randomlist) '

In [ ]:
""" pd_oversampling=pd_train
for i in randomlist:
  pd_oversampling = pd_oversampling.append(pd_train_fireyes.iloc[i], ignore_index=True) """

' pd_oversampling=pd_train\nfor i in randomlist:\n  pd_oversampling = pd_oversampling.append(pd_train_fireyes.iloc[i], ignore_index=True) '

In [ ]:
#undersampling
import random
randomlist = []
for i in range(0,int(len(pd_train_fireyes))):
  n = random.randint(1,len(pd_train_fireyes))
  randomlist.append(n)

print(randomlist)

[2938, 4533, 3313, 3539, 766, 2360, 2013, 3839, 3406, 4868, 4544, 6995, 1716, 7071, 6390, 5927, 1495, 6316, 1832, 4488, 2282, 1123, 7250, 5441, 4095, 590, 153, 370, 5158, 2602, 6716, 4479, 954, 2991, 3852, 524, 1106, 5333, 5718, 1040, 2691, 4740, 2290, 7584, 783, 5797, 4165, 2924, 2556, 1616, 7623, 6396, 73, 5419, 6881, 3730, 1197, 4718, 88, 4585, 3253, 6695, 2333, 7082, 2820, 2841, 6392, 2006, 2167, 7162, 4936, 3498, 6274, 2058, 3429, 136, 3763, 5748, 3094, 5757, 4741, 4500, 893, 6782, 3842, 1927, 5010, 92, 1326, 3213, 1463, 5111, 8, 754, 6120, 3304, 2281, 3897, 736, 2244, 1512, 6847, 2571, 6587, 261, 3705, 4753, 1408, 3333, 3221, 5745, 6473, 2977, 7043, 3602, 2083, 1398, 337, 1913, 2512, 2439, 2340, 3905, 844, 5037, 1618, 2658, 4487, 5499, 669, 1630, 1248, 2789, 5083, 7316, 5095, 2980, 4138, 4626, 2776, 7027, 1116, 5702, 498, 5126, 901, 1670, 3627, 4506, 7309, 2470, 6356, 4979, 3384, 2254, 1342, 1439, 751, 1928, 6418, 2889, 1957, 4069, 7582, 7623, 4241, 370, 919, 3978, 7397, 1885, 46

In [ ]:
new_no_downsampling=pd.DataFrame()
new_no_downsampling = new_no_downsampling.append(pd_train_fireno.iloc[randomlist[0:len(randomlist)]], ignore_index=True)
new_no_downsampling

dt_of_fr fr_yn  bldng_cnt  ...  ahsm_dstnc  no_tbc_zn_dstnc  bldng_cnt_in_50m
0            3     N          2  ...         518             2530                 0
1            0     N          2  ...        7074              116                 1
2            0     N          4  ...        1412             5477                 0
3            1     N          2  ...         152               35                21
4            3     N          2  ...         467              467                 0
...        ...   ...        ...  ...         ...              ...               ...
7652         2     N          2  ...        6200              427                 0
7653         1     N          4  ...         163              262                 0
7654         1     N          5  ...       15603             3805                 0
7655         2     N          2  ...         102              414                15
7656         1     N          3  ...        6633             4379                36

[7657 rows x 160 columns]

In [ ]:
#yes 비중 반, no 비중 반으로 맞춘 downsampling
#no 비중을 줄이는 방법은 빈도주의적 방법으로 랜덤하게 값 제거함.
pd_newtrain=pd_train_fireyes
pd_newtrain = pd_newtrain.append(new_no_downsampling, ignore_index=True)
pd_newtrain

dt_of_fr fr_yn  bldng_cnt  ...  ahsm_dstnc  no_tbc_zn_dstnc  bldng_cnt_in_50m
0             3     Y          3  ...       11322               88                 0
1             3     Y          1  ...       12451               72                14
2             1     Y          1  ...       11690              206                 0
3             0     Y          2  ...       12101               18                 7
4             2     Y          3  ...       11517               11                 0
...         ...   ...        ...  ...         ...              ...               ...
15309         2     N          2  ...        6200              427                 0
15310         1     N          4  ...         163              262                 0
15311         1     N          5  ...       15603             3805                 0
15312         2     N          2  ...         102              414                15
15313         1     N          3  ...        6633             4379                36

[15314 rows x 160 columns]

In [ ]:
#타겟을 1과 0으로 바꿔줌
for i in range(len(pd_newtrain['fr_yn'])):
  if pd_newtrain['fr_yn'][i]=='Y':
    pd_newtrain['fr_yn'][i]=1
  else:
    pd_newtrain['fr_yn'][i]=0

In [ ]:
pd_newtrain

dt_of_fr fr_yn  bldng_cnt  ...  ahsm_dstnc  no_tbc_zn_dstnc  bldng_cnt_in_50m
0             3     1          3  ...       11322               88                 0
1             3     1          1  ...       12451               72                14
2             1     1          1  ...       11690              206                 0
3             0     1          2  ...       12101               18                 7
4             2     1          3  ...       11517               11                 0
...         ...   ...        ...  ...         ...              ...               ...
15309         2     0          2  ...        6200              427                 0
15310         1     0          4  ...         163              262                 0
15311         1     0          5  ...       15603             3805                 0
15312         2     0          2  ...         102              414                15
15313         1     0          3  ...        6633             4379                36

[15314 rows x 160 columns]

# **모델링 시작**

In [ ]:
pd_newtrain_target=pd_newtrain['fr_yn']
pd_newtrain_설명변수=pd_newtrain.drop(['fr_yn'],axis=1) 

In [ ]:
df1_x_Train, df1_x_test, df1_y_Train, df1_y_test = train_test_split(pd_newtrain_설명변수, pd_newtrain_target, test_size=0.2, random_state=None, stratify=pd_newtrain_target)
df1_x_train, df1_x_valid, df1_y_train, df1_y_valid = train_test_split(df1_x_Train ,df1_y_Train, test_size=(2/8), random_state=None, stratify=df1_y_Train)

### XGBoosting

In [ ]:
import xgboost as xgb
from sklearn.metrics import mean_squared_error

In [ ]:
dtrain = xgb.DMatrix(df1_x_train, label=df1_y_train.values)
dvalid = xgb.DMatrix(df1_x_valid, label=df1_y_valid.values)
dtest = xgb.DMatrix(df1_x_test, label=df1_y_test.values)


# max_depth = 3, 학습률은 0.1, 예제가 이진분류이므로 목적함수(objective)는 binary:logistic(이진 로지스틱)
# 오류함수의 평가성능지표는 logloss
# 부스팅 반복횟수는 400
# 조기중단을 위한 최소 반복횟수는 100

params = {'max_depth' : 3,
         'eta' : 0.1, 
         'objective' : 'binary:logistic',
         'eval_metric' : 'logloss',
         'early_stoppings' : 100 }

num_rounds = 400

# train 데이터 세트는 'train', evaluation(test) 데이터 세트는 'eval' 로 명기
wlist = [(dtrain, 'train'), (dvalid, 'valid'), (dtest,'test')]
# 하이퍼 파라미터와 early stopping 파라미터를 train() 함수의 파라미터로 전달
xgb_model = xgb.train(params = params, dtrain=dtrain, num_boost_round=num_rounds, evals=wlist)




In [ ]:
pred_probs = xgb_model.predict(dvalid)
print('predict() 수행 결과값을 10개만 표시, 예측 확률 값으로 표시됨')
print(np.round(pred_probs[:10], 3))


def XGB_find_threshold(y_valid, y_valid_p, threshold1):
    table = pd.DataFrame()

    for i in threshold1:
        pred_Y = cut_off(y_valid_p, i)
        cfmat = confusion_matrix(y_valid, pred_Y)
        table.loc[i, 'accuracy'] = accuracy(cfmat)
        table.loc[i,  'recall'] = recall(cfmat)
        table.loc[i, 'precision'] = precision(cfmat)
        table.loc[i, 'f1_score'] = f1_score(cfmat)
    
    table.index.name='threshold'
    table.columns.name='performance'
    best = table['f1_score'].idxmax()
    best_per = pd.DataFrame(table.loc[best]) 

predict() 수행 결과값을 10개만 표시, 예측 확률 값으로 표시됨
[0.997 0.011 0.266 0.783 0.183 0.424 0.001 0.399 0.513 0.911]


In [ ]:
from sklearn.metrics import confusion_matrix
table = pd.DataFrame()
threshold = np.arange(0, 1, 0.01)

for i in threshold:
    pred_Y = cut_off(pred_probs, i)
    cfmat = confusion_matrix( df1_y_valid.tolist(), pred_Y.tolist()) 
    table.loc[i, 'accuracy'] = accuracy(cfmat)
    table.loc[i,  'recall'] = recall(cfmat)
    table.loc[i, 'precision'] = precision(cfmat)
    table.loc[i, 'f1_score'] = f1_score(df1_y_valid.tolist(), pred_Y)
def cut_off(y, threshold):
    Y = y.copy() 
    Y[Y >= threshold]=1
    Y[Y < threshold]=0
    return(Y.astype(int))


table.index.name='threshold'
table.columns.name='performance'
best = table['f1_score'].idxmax()
best_per = pd.DataFrame(table.loc[best])

best_per

0.46
performance          
accuracy     0.800522
recall       0.847812
precision    0.774463
f1_score     0.809479

In [ ]:
# 예측 확률이 best보다 크면 1, 그렇지 않으면 0으로 예측값 결정해 리스트 객체인 preds에 저장
pred_probs_test = xgb_model.predict(dtest)
preds = [ 1 if x > best else 0 for x in pred_probs_test]
print('예측값 10개만 표시: ', preds[:10])

예측값 10개만 표시:  [0, 1, 0, 1, 0, 1, 0, 1, 0, 1]


In [ ]:
# 혼동행렬, 정확도, 정밀도, 재현율, F1, AUC 불러오기

from sklearn.metrics import f1_score

def get_clf_eval(y_test, y_pred):
    confusion = confusion_matrix(y_test, y_pred)
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    F1 = f1_score(y_test, y_pred)
    AUC = roc_auc_score(y_test, y_pred)
    print('오차행렬:\n', confusion)
    print('\n정확도: {:.4f}'.format(accuracy))
    print('정밀도: {:.4f}'.format(precision))
    print('재현율: {:.4f}'.format(recall))
    print('F1: {:.4f}'.format(F1))
    print('AUC: {:.4f}'.format(AUC))


In [ ]:
get_clf_eval(df1_y_test.tolist(), preds)

오차행렬:
 [[1135  396]
 [ 285 1247]]

정확도: 0.7777
정밀도: 0.7590
재현율: 0.8140
F1: 0.7855
AUC: 0.7777


In [ ]:
from xgboost import plot_importance
import matplotlib.pyplot as plt
%matplotlib inline

fig, ax = plt.subplots(figsize=(10, 12))
plot_importance(xgb_model, ax=ax)

In [ ]:
################################################################################
##############로지스틱, 랜덤 포레스트, 의사결정트리는 부도예측모델 동일하게 사용
################################################################################

In [ ]:
# y예측값 구하기 - 1: y >= threshold, 0: y < threshold
def cut_off(y, threshold):
    Y = y.copy() 
    Y[Y >= threshold]=1
    Y[Y < threshold]=0
    return(Y.astype(int))

# 정확도
def accuracy(cfmat) :
    acc=(cfmat[0,0]+cfmat[1,1]) / np.sum(cfmat) ## accuracy
    return(acc)

# 재현률
def recall(cfmat) :
      rec=(cfmat[1,1]) / (cfmat[1,0] + cfmat[1,1]) ## recall
      return(rec)

# 정밀도
def precision(cfmat) :
      preci=(cfmat[1,1]) / (cfmat[0,1] + cfmat[1,1])  ## precision
      return(preci)
      
#f1score val
def f1_score(cfmat):
    f1_scoreval=2*precision(cfmat)*recall(cfmat)/(precision(cfmat)+recall(cfmat))
    return f1_scoreval

### 2.0.2 로지스틱 threshold 구하기 

In [ ]:
# Validation에 적합하여 threshold 구하기
def Logit_find_threshold(y_valid, y_valid_p, x_valid, threshold1):
    table = pd.DataFrame()

    for i in threshold1:
        pred_Y = cut_off(y_valid_p, i)
        cfmat = confusion_matrix(y_valid, pred_Y)
        table.loc[i, 'accuracy'] = accuracy(cfmat)
        table.loc[i,  'recall'] = recall(cfmat)
        table.loc[i, 'precision'] = precision(cfmat)
        table.loc[i, 'f1_score'] = f1_score(cfmat)
    
    table.index.name='threshold'
    table.columns.name='performance'
    best = table['f1_score'].idxmax()
    best_per = pd.DataFrame(table.loc[best]) 

    
    return best, best_per, table

def Logit_best_threshold(valid, y_test, y_test_p, x_test):
    
    best_threshold = valid ['f1_score'].idxmax()
    pred_Y = cut_off(y_test_p, best_threshold )
    cfmat1 = confusion_matrix(y_test, pred_Y)

    table1 = pd.DataFrame()
    table1.loc[1,'accuracy'] = accuracy(cfmat1)
    table1.loc[1,'recall'] = recall(cfmat1)
    table1.loc[1,'precision'] = precision(cfmat1)
    table1.loc[1,'f1_score'] = f1_score(cfmat1)

    table1.columns.name='performance'
    table1.index = [best_f1_score]
    table1.index.name='f1_score'
  
    return best_f1_score, cfmat1, table1

### 2.0.3 의사결정나무 depth 구하기

In [ ]:
# 의사결정나무의 파라메터: max_depth 깊이, max_features 각 노드에서 분할에 사용할 특성의 최대 수 
# Validation에 적합하여 max_depth와 max_features 구하기
#  max_depth와 max_features 구하기
def DT_find_depth_feature(x_train, y_train, x_valid, y_valid, depth,  feature):

  accur=[]; rec=[]; preci=[]; ep=[]; dep=[]; nf=[];
  for i in depth:
    for j in  feature:
      model = DecisionTreeClassifier(criterion="entropy", max_depth = i, max_features=j)
      result = model.fit(x_train, y_train)
      pred_Y = result.predict(x_valid)
      cfmat1 = confusion_matrix(y_valid, pred_Y)
      accur.append(accuracy(cfmat1))
      rec.append(recall(cfmat1))
      preci.append(precision(cfmat1))
      ep.append(f1_score(cfmat1))
      dep.append(i)
      nf.append(j)
      
  table4 = pd.DataFrame()
  table4['depth'] = dep
  table4['max_feature'] = nf
  table4['accuracy'] = accur
  table4['recall'] = rec
  table4['precision'] = preci
  table4['f1_score'] = ep
  table4.columns.name='performance'
  best_ind = table4['f1_score'].idxmax()
  best_per = pd.DataFrame(table4.loc[best_ind]) 
  best = best_per.iloc[:2, :]
  return  best, best_per, table4


# 결정된 max_depth와 max_features로 test data적합
def DT_best_depth(best, x_train, y_train, x_test, y_test):
    
    best_dep = round(best.iloc[0, 0])
    best_feat = round(best.iloc[1, 0])

    model1 = DecisionTreeClassifier(criterion="entropy", max_depth = best_dep, max_features = best_feat)
    best_model = model1.fit(x_train, y_train)
    pred_Y = best_model.predict(x_test)
    cfmat2 = confusion_matrix(y_test, pred_Y)


    table4 = pd.DataFrame()
    table4.loc[1,'accuracy'] = accuracy(cfmat2)
    table4.loc[1,'recall'] = recall(cfmat2)
    table4.loc[1,'precision'] = precision(cfmat2)
    table4.loc[1,'f1_score'] = f1_score(cfmat2)
    table4.columns.name='performance'
  
    return best_dep, best_feat, best_model, cfmat2, table4

### 2.0.4 랜덤포레스트 파라메타 구하기
- n_estimators: 트리의 갯수 지정
- max_features: 최적의 분할을 위해 고려할 최대 feature 수
- max_depth: 트리 최대 깊이
- max_leaf_nodes: 리프노드의 최대 개수 
- min_samples_split: 노드 분할하기 위한 최소한의 샘플데이터 수
- min_samples_leaf: 리프노드가 되기 위해 필요한 최소한의 샘플 데이터 수 

In [ ]:
# Validation에 적합하여 n_estimators와 max_depth 구하기
# n_estimators와 max_depth 구하기
def RF_find_N_depth(x_train, y_train, x_valid, y_valid, n_estimators,  max_depth):

  accur=[]; rec=[]; preci=[]; ep=[]; N=[]; dep=[];
  for i in n_estimators:
    for j in  max_depth:
      model = RandomForestClassifier(criterion="entropy", max_features='auto', n_estimators = i, max_depth=j)
      result = model.fit(x_train, y_train)
      pred_Y = result.predict(x_valid)
      cfmat1 = confusion_matrix(y_valid, pred_Y)
      accur.append(accuracy(cfmat1))
      rec.append(recall(cfmat1))
      preci.append(precision(cfmat1))
      ep.append(f1_score(cfmat1))
      dep.append(i)
      N.append(j)
      
  table5 = pd.DataFrame()
  table5['n_estimators'] = N
  table5['depth'] = dep
  table5['accuracy'] = accur
  table5['recall'] = rec
  table5['precision'] = preci
  table5['f1_score'] = ep
  table5.columns.name='performance'
  best_ind = table5['f1_score'].idxmax()
  best_per = pd.DataFrame(table5.loc[best_ind]) 
  best = best_per.iloc[:2, :]
  return  best, best_per, table5

# 결정된  n_estimators, max_depth로 test data적합
def RF_best_N_depth(best, x_train, y_train, x_test, y_test):
    
    best_N = round(best.iloc[0, 0])
    best_dep = round(best.iloc[1, 0])

    model2 = RandomForestClassifier(criterion="entropy", max_features='auto', n_estimators =  best_N, max_depth = best_dep)
    best_model = model2.fit(x_train, y_train)
    pred_Y = best_model.predict(x_test)
    cfmat2 = confusion_matrix(y_test, pred_Y)


    table4 = pd.DataFrame()
    table4.loc[1,'accuracy'] = accuracy(cfmat2)
    table4.loc[1,'recall'] = recall(cfmat2)
    table4.loc[1,'precision'] = precision(cfmat2)
    table4.loc[1,'f1_score'] = f1_score(cfmat2)
    table4.columns.name='performance'
  
    return best_N, best_dep, best_model, cfmat2, table4

### 로지스틱

In [ ]:
Logit_model_df1 = LogisticRegression(fit_intercept=False)
df1_y_train=df1_y_train.astype('int')
Logit_result_df1 = Logit_model_df1.fit(df1_x_train, df1_y_train)

In [ ]:
print(df1_x_train.shape, df1_y_train.shape)

(9188, 159) (9188,)


In [ ]:
Logit_y_valid_p_df1 = Logit_result_df1.predict_proba(df1_x_valid)[:,1]
Logit_y_valid_p_df1

array([1.        , 0.41531521, 0.61631259, ..., 0.8056667 , 0.43421438,
       0.5628806 ])

In [ ]:
from sklearn.metrics import confusion_matrix
table = pd.DataFrame()
threshold = np.arange(0, 1, 0.01)

for i in threshold:
    pred_Y = cut_off(Logit_y_valid_p_df1, i)
    cfmat = confusion_matrix( df1_y_valid.tolist(), pred_Y.tolist()) 
    table.loc[i, 'accuracy'] = accuracy(cfmat)
    table.loc[i,  'recall'] = recall(cfmat)
    table.loc[i, 'precision'] = precision(cfmat)
    table.loc[i, 'f1_score'] = f1_score(cfmat)
def cut_off(y, threshold):
    Y = y.copy() 
    Y[Y >= threshold]=1
    Y[Y < threshold]=0
    return(Y.astype(int))


table.index.name='threshold'
table.columns.name='performance'
best = table['f1_score'].idxmax()
best_per = pd.DataFrame(table.loc[best])

best_per

0.36
performance          
accuracy     0.604962
recall       0.927498
precision    0.563716
f1_score     0.701235

In [ ]:
# Test에 적합한 결과 

Logit_y_test_p_df1 = Logit_result_df1.predict_proba(df1_x_test)[:,1]
pred_Y_test = cut_off(Logit_y_test_p_df1, best)

import sklearn.metrics as metrics
metrics.f1_score(df1_y_test.tolist(),pred_Y_test.tolist() )


0.6996273291925464

In [ ]:
# 회귀 계수 값 
Logit_coef_df1_1 = pd.Series(Logit_result_df1.coef_[0], index = df1_x_train.columns)
Logit_coef_df1 = Logit_coef_df1_1.sort_values(ascending=False)
round(Logit_coef_df1,5)

ttl_ar                  0.00003
ahsm_dstnc              0.00003
bldng_ar                0.00002
sft_emrgnc_bll_dstnc    0.00001
gas_engry_us_201611     0.00001
                         ...   
no_tbc_zn_dstnc        -0.00002
hm_cnt                 -0.00002
tbc_rtl_str_dstnc      -0.00003
fr_sttn_dstnc          -0.00005
fr_wthr_fclt_dstnc     -0.00007
Length: 159, dtype: float64

In [ ]:
round(np.exp(Logit_coef_df1),5)

ttl_ar                  1.00003
ahsm_dstnc              1.00003
bldng_ar                1.00002
sft_emrgnc_bll_dstnc    1.00001
gas_engry_us_201611     1.00001
                         ...   
no_tbc_zn_dstnc         0.99998
hm_cnt                  0.99998
tbc_rtl_str_dstnc       0.99997
fr_sttn_dstnc           0.99995
fr_wthr_fclt_dstnc      0.99993
Length: 159, dtype: float64

### 의사결정나무

In [ ]:
DT_model_df1 = DecisionTreeClassifier(criterion="entropy")
DT_result_df1 = DT_model_df1.fit(df1_x_train, df1_y_train)

In [ ]:
DT_result_df1.score

<bound method ClassifierMixin.score of DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='entropy',
                       max_depth=None, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=None, splitter='best')>

In [ ]:
# valid set을 이용하여 max_depth와 max_feature 결정 
DT_depth = np.arange(3, 10, 1) 
DT_feature = np.arange(1, 10, 1) 
DT_best_df1, DT_best_per_df1, DT_valid_df1 = DT_find_depth_feature(df1_x_train.values.tolist(), df1_y_train.values.tolist(), df1_x_valid.values.tolist(), df1_y_valid.values.tolist(), DT_depth,  DT_feature)

In [ ]:
DT_valid_df1

performance  depth  max_feature  accuracy    recall  precision  f1_score
0                3            1  0.513222  0.029393   0.900000  0.056926
1                3            2  0.648710  0.342913   0.882353  0.493885
2                3            3  0.650669  0.346832   0.883527  0.498124
3                3            4  0.659158  0.372959   0.871756  0.522415
4                3            5  0.694744  0.487263   0.832589  0.614751
..             ...          ...       ...       ...        ...       ...
58               9            5  0.693111  0.478772   0.837714  0.609310
59               9            6  0.696376  0.504246   0.818664  0.624091
60               9            7  0.714006  0.551274   0.817038  0.658346
61               9            8  0.715312  0.670803   0.736201  0.701982
62               9            9  0.711721  0.638798   0.747706  0.688975

[63 rows x 6 columns]

In [ ]:
DT_best_per_df1

11
performance          
depth        4.000000
max_feature  3.000000
accuracy     0.662749
recall       0.963423
precision    0.601550
f1_score     0.740648

In [ ]:
DT_best_df1

11
performance     
depth        4.0
max_feature  3.0

In [ ]:
# test set을 이용하여 예측 
DT_best_dep_df1, DT_best_feat_df1, DT_best_model_df1, DT_cfmat_df1, DT_test_df1 = DT_best_depth(DT_best_df1, df1_x_train.values.tolist(), df1_y_train.values.tolist(), df1_x_test.values.tolist(), df1_y_test.values.tolist())





In [ ]:
print(DT_best_dep_df1, DT_best_feat_df1)

4 3


In [ ]:
DT_cfmat_df1

array([[1452,   79],
       [ 959,  573]])

In [ ]:
DT_test_df1

performance  accuracy    recall  precision  f1_score
1            0.661117  0.374021   0.878834  0.524725

In [ ]:
# 의사결정트리 시각화를 위한 작업
# 트리 시각화를 위한 export_graphviz 모듈 임포트 (이를 위해 graphviz 별도 설치 필요)
from sklearn.tree import export_graphviz
import pydotplus  # graphviz의 dot language 와의 인터페이스를 제공하는 패키지 임포트
from IPython.display import Image  # IPython의 display와 관련된 Public API

x_list = list(df1_x_test.columns)
y_list = list(df1_y_test.drop_duplicates(inplace=False))

# export_graphviz() : 의사결정트리에 대한 graphviz dot data를 생성하는 함수
dot_data = export_graphviz(DT_best_model_df1, out_file=None, feature_names=x_list,
                          class_names=str(y_list), filled=True, rounded=True, special_characters=True)
# matplotlib.rc('font', family='AppleGothic')
graph = pydotplus.graph_from_dot_data(dot_data)  # graphviz의 dot data로부터 트리 그래프 생성

# 트리 그래프를 위한 png 이미지 생성 및 출력
Image(graph.create_png())

### 랜덤포레스트

In [ ]:
# valid set을 이용하여 max_dept, max_feature 결정 
RF_N = np.arange(5, 21, 1) 
RF_depth = np.arange(3, 10, 1) 
RF_best_df1, RF_best_per_df1, RF_valid_df1 = RF_find_N_depth(df1_x_train.values.tolist(), df1_y_train.values.tolist(), df1_x_valid.values.tolist(), df1_y_valid.values.tolist(), RF_N, RF_depth)

In [ ]:
RF_valid_df1

performance  n_estimators  depth  accuracy    recall  precision  f1_score
0                       3      5  0.689847  0.450033   0.864492  0.591924
1                       4      5  0.680052  0.413455   0.885315  0.563669
2                       5      5  0.735227  0.573481   0.847490  0.684067
3                       6      5  0.744368  0.670150   0.786810  0.723810
4                       7      5  0.736859  0.611365   0.816042  0.699029
..                    ...    ...       ...       ...        ...       ...
107                     5     20  0.729024  0.563031   0.842620  0.675020
108                     6     20  0.739798  0.564337   0.869215  0.684356
109                     7     20  0.748612  0.630960   0.824936  0.715026
110                     8     20  0.756448  0.634879   0.838654  0.722677
111                     9     20  0.777669  0.720444   0.813422  0.764115

[112 rows x 6 columns]

In [ ]:
RF_best_per_df1

62
performance            
n_estimators   9.000000
depth         13.000000
accuracy       0.778648
recall         0.747877
precision      0.796799
f1_score       0.771563

In [ ]:
RF_best_df1

62
performance       
n_estimators   9.0
depth         13.0

In [ ]:
# test set을 이용하여 예측 
RF_best_N_df1, RF_best_dep_df1, RF_best_model_df1, RF_cfmat_df1, RF_test_df1 = RF_best_N_depth(RF_best_df1, df1_x_train.values.tolist(), df1_y_train.values.tolist(), df1_x_test.values.tolist(), df1_y_test.values.tolist())

In [ ]:
 RF_cfmat_df1

array([[1233,  298],
       [ 426, 1106]])

In [ ]:
 RF_test_df1 

performance  accuracy    recall  precision  f1_score
1             0.76363  0.721932   0.787749  0.753406

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

RF_importances_values = RF_best_model_df1.feature_importances_
RF_importances = pd.Series(RF_importances_values, index = df1_x_train.columns)
RF_top20 = RF_importances.sort_values(ascending=False)[:20]

plt.figure(figsize=(8,6))
plt.title('Top 20 Feature Importances')
sns.barplot(x=RF_top20, y=RF_top20.index)
plt.show()